In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None) 
sns.set_palette("Set2")
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
sample = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
text_file = open("/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt", "r")
description = text_file.read()

In [ ]:
train.shape

In [ ]:
train.info()

The data file contains three types: int64, object, float types. 
We can firstly check each type of attribute, then we want to deal with NaN value. If the type is numeric attribute, we fill it with 0. If the type is categorical value, we fill it with "missing". If there are significant missing value (90% are null) in a feature, we drop that feature. 
## Categorical attribute
 Firstly, working with categorical attributes, lets see what information the data provides.

In [ ]:
objectscol = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition']  #<---train.describe(include =np.object).columns

infotable = train.describe(include =np.object)
infotable

For each attributes in the info table, we firstly focusing on the missing value. 
- step 1: remove the 90% null value attributes. 

In [ ]:
for attribute in objectscol:
    x = infotable.loc['count'][attribute]/1460
    if x < 0.1:print(attribute)

- step 2: fill the rest of object's null value with "missing".

In [ ]:
train = train.drop(["Alley","PoolQC","MiscFeature"], axis = 1)
train.shape

In [ ]:
objectcol = train.describe(include = np.object).columns
train[objectcol] = train[objectcol].fillna("missing")

![](http://) Take an example of *Fence* attribute, we noticed the majority is filled with missing value.

In [ ]:
sns.countplot(train["Fence"])

In [ ]:
len(objectcol)

Take an overview of all the catogrical attributes.

In [ ]:
fig = plt.figure(figsize=(10,50))
fig.subplots_adjust(hspace=0.8, wspace=0.8)

for i in range(1, 41):
    ax = fig.add_subplot(20, 2, i)
    g = sns.countplot(train[objectcol[i-1]])
    plt.xticks(rotation=45)
    

> ## Numeric Attributes
- Float64
- Int64
#### Now, move to the numeric attributes:
- step 1: find all the numeric columns.
- step 2: check the nan values percentatge, if it is over 90% are missing, then we delete this attributes
- step 3: fill the rest attributes naa with 0 if nothing indicated .

Float64 is already using 0 represnet NANs, we simply check the 0 situation, same applt for Int64.

In [ ]:
flotcol = train.describe(include = np.float64).columns
infortable2 = train.describe(include = np.float64)
for attribute in flotcol:
    x = infortable2.loc['count'][attribute]/1460
    if x < 0.1:
        print("found 90% null attribute:{}",attribute)
    else:
        print("no attribute found with over 90% empty")
        pass
        #nothing happend
train[flotcol]=train[flotcol].fillna(0)


In [ ]:
fig = plt.figure(figsize=(15,5))
fig.subplots_adjust(hspace=0.8, wspace=0.8)
for i in range(1, 4):
    ax = fig.add_subplot(1, 3, i)
    sns.distplot(train[flotcol[i-1]])

In [ ]:
# INT64 attribiutes
intcol = train.describe(include = np.int64).columns
infotable3 = train.describe(include = np.int64)
infotable3

In [ ]:
for attribute in intcol:
    x = infotable3.loc['count'][attribute]/1460
    if x < 0.1:
        print("found 90% null attribute:{}",attribute)
    else:
        pass
        #nothing happend
print("finished")

In [ ]:
len(intcol)

 #### Plot the overview of int64 attributes

In [ ]:
#sns.distplot(train["MSSubClass"])
fig = plt.figure(figsize=(15,15))
fig.subplots_adjust(hspace=0.8, wspace=0.8)
for i in range(1, 36):
    ax = fig.add_subplot(7, 5, i)
    sns.distplot(train[intcol[i-1]])



## Relation exploration
Take a closer look at the relationship between year sold and housing price: 

In [ ]:
sns.violinplot(y = "SalePrice", x = "YrSold", data = train)

- The range of the price are not changing much alone five years

> #### There are many attributes (80)! I will frist make a feature selection then draw the correlation graphs to seek correlations.
## Feature Importance check[](http://)
> #### Many ways are possible for feature selection, hereby I choose to use   *Extra Trees Ckaasifier*, which is an ensemble method could plot feature importance. 

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
target = train['SalePrice']
features = train.drop(columns = ['SalePrice','Id'])
#only possible for numeric features, so we here only use numeric
non_obj_features = features.describe(exclude = np.object).columns
features = features[non_obj_features]
beforefs = features.columns
print("before feature selection:{} number of features.".format(len(non_obj_features)))
#create a classifer for feature selection
forest  = ExtraTreesRegressor(n_estimators = 50)
forest = forest.fit(features, target)
importance_table = pd.Series(forest.feature_importances_, index=features.columns)
importance_table = importance_table.sort_values(ascending = False)

plt.figure(figsize=(16,8))
ax = sns.barplot(x = importance_table.index[:10], y = importance_table.values[:10], palette="BuGn_r")
ax.set_title('Feature Importance')

select_feature = importance_table.index[:10]
#importance_table

In [ ]:
select_feature 

From the forest regressor we have select 10 most important features, they are  * 'OverallQual', 'GarageCars', 'GrLivArea', 'FullBath', 'TotalBsmtSF','GarageArea', 'YearBuilt', 'Fireplaces', '1stFlrSF', 'BsmtFinSF1' *

- Apparently "OverallQual" is the most relevant attribute in terms of the sale price. This is indicateing the Overall material and finish quality of a house. 
- "GarageCars" is the size of garage in car capacity, ranging from 0 - 4 in this case. 


In [ ]:
fig = sns.PairGrid(train[select_feature[0:5]])
fig.map_offdiag(plt.scatter)
fig.map_diag(sns.distplot, bins=30)

We noticed from selected features part I:
- Most of GrLivArea (Above ground living area square feet) is between 0-4000. 
- The lower the GrliveArea corresponse with less number of fullBath (0,1,2,3)
- The OverallQual( Rates the overall material and finish of the house, which is range from 1-10) shows houese build from 1950 are mostly above or equal to level 5- average. 

Lets move to the second five other attributes have selected.[](http://)

In [ ]:
fig = sns.PairGrid(train[select_feature[5:]])
fig.map_offdiag(plt.scatter)
fig.map_diag(sns.distplot, bins=30);

We noticed from selected features part II:
- TotalBsmtSF (Total square feet of basement area) is strongly increasing as well as the 1stFLrSF(First Floor square feet). And nomrally 1st floor sf is just bigger than basement. 
- BsmtFinSF1 (Basement finihsed area type1 in square feet) is correlated with TotalBsmtSF and 1stFLrSF

After Checked the ten most correlated features according to ExtraTreesRegressor, we noticed there are few correlations exist between basement data. Next, we will check the correlation from features with target ->sale price.

## More pattern discovery
#### 1.The correlation between sale price and selected 10 features

In [ ]:
fig = plt.figure(figsize=(10,20))
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i in range(1, 11):
    ax = fig.add_subplot(5, 2, i)
    g = sns.scatterplot(x = train[select_feature[i-1]], y="SalePrice", data=train)
    g.set_xlabel(g.get_xlabel(),fontsize= 20)
    g.set_ylabel(g.get_ylabel(),fontsize= 15)
    plt.xticks(rotation=45)

> #### 2.Sale price with all the categorical features, if we noticed there some attributes exist patterns,we can encode it later. 
The way I present is Violin Graph, there is a blog nicely describes how we can interpret: https://mode.com/blog/violin-plot-examples

In [ ]:
fig = plt.figure(figsize=(20,150))
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i in range(1, 41):
    ax = fig.add_subplot(20, 2, i)
    g = sns.violinplot(x = train[objectcol[i-1]], y="SalePrice", data=train)
    g.set_xlabel(g.get_xlabel(),fontsize= 20)
    g.set_ylabel(g.get_ylabel(),fontsize= 15)
    plt.xticks(rotation=45)

In [ ]:
len(objectcol)

Now I am going to fit my selected feature with a machine leanring algroithm and born a predictive model. However, I just realize I still have null vlue exsit on my test data! Lets take a look..

In [ ]:
test[select_feature].info()

In [ ]:
#Checking misisng valus
missing_val_count_by_column = (test[select_feature].isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
test[select_feature].info()

In [ ]:
missing_val_count_by_column = (test[select_feature].isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

- A simple way to replace the missing numeric value is replace with the average number. 

In [ ]:
missing_cols =missing_val_count_by_column[test[select_feature].isnull().sum()>0].index
for i in range(0,len(missing_cols)):   
    fillwith = test[missing_cols[i]].mean()
    test[missing_cols[i]].fillna(value=fillwith, inplace  = True)

> #### Prepare categorical value data before running machine leanring model:

In [ ]:
X = train.drop(columns =['Id','SalePrice'])

In [ ]:
print("There are {} features in used. ".format(len(X.columns)))

In [ ]:
X = pd.get_dummies(X)

In [ ]:
print("After dummies applied, There are {} features in used. ".format(len(X.columns)))

In [ ]:
X_test = test.drop(columns =['Id'])
X_test = pd.get_dummies(X_test)

In [ ]:
missing_val_count_by_column = (X_test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
#missing_cols =missing_val_count_by_column[X_test.isnull().sum()>0].index
for i in range(0,len(missing_cols)):   
    fillwith = X_test[missing_cols[i]].mean()
    X_test[missing_cols[i]].fillna(value=fillwith, inplace  = True)

- Apparently in the test file there contains more columns than in the train file. So I made them the same.

In [ ]:
X_test =X_test.drop(columns =['PoolQC_Gd', 'Alley_Pave', 'Alley_Grvl', 'MiscFeature_Shed', 'MiscFeature_Othr', 'PoolQC_Ex', 'MiscFeature_Gar2'])

In [ ]:
X = X[X_test.columns]

In [ ]:
missing_val_count_by_column = (X_test.isnull().sum())
missing = missing_val_count_by_column[missing_val_count_by_column > 0]
missing_cols =missing_val_count_by_column[X_test.isnull().sum()>0].index
for i in range(0,len(missing)):   
    fillwith = X_test[missing_cols[i]].mean()
    print(fillwith)
    X_test[missing_cols[i]] = X_test[missing_cols[i]].fillna(value=fillwith)

## ML 1: Randomforest

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# forest = RandomForestRegressor(n_estimators=100)
# forest.fit(X,target)
# result = forest.predict(X_test)
# handin = pd.DataFrame({'Id': test.Id,'SalePrice': result})
# handin.to_csv('submission.csv', index=False)

### Grid search on the forest classifer, try to improve the performance. 


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


parameters  = { 
    'n_estimators': [50,100,200,300,700],
    'max_features': ['auto', 'sqrt', 'log2']
}
forest = RandomForestRegressor()
#apply grid search cv method on the forest and parameters.
gs_forest = GridSearchCV(forest, parameters, cv = 5)
gs_forest.fit(X, target)

In [ ]:
print("Best score: {}, \nBest params: {}".format(gs_forest.best_score_,gs_forest.best_estimator_))

In [ ]:
#Call predict on the estimator with the best found parameters.
result = gs_forest.predict(X_test)
handin = pd.DataFrame({'Id': test.Id,'SalePrice': result})
handin.to_csv('submission.csv', index=False)

## ML 2: PCA + SVR
1. Apply standard scalar on both train/test data.
2. Apply PCA for dimesion reduction.
3. Applly Support Vector regressor.

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from sklearn.svm import SVR

# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)
# X_test = scaler.transform(X_test)
# pca = PCA(n_components=10)
# pca.fit(X)
# X = pca.transform(X)
# X_test = pca.transform(X_test)
# clf = SVR(gamma='scale', C=1.0, epsilon=0.2)
# clf.fit(X,target)
# result = clf.predict(X_test)
# handin = pd.DataFrame({'Id': test.Id,'SalePrice': result})
# handin.to_csv('submission.csv', index=False)
# print("score:",0.4)